In [2]:
from world_machine.train.scheduler import UniformScheduler, ChoiceScheduler
from world_machine.train.stages import StateSaveMethod

from world_machine_experiments.toy1d import Dimensions

In [2]:
n_epoch = 100

n_variation = 0

for n_segment in [1,2]:
    fast_forward_choices = []
    if n_segment >1:
        fast_forward_choices = [True, False]

    for fast_forward in fast_forward_choices:
        for stable_state_epochs in [1, 2, 5]:
            for check_input_masks in [True, False]:
                for state_save_method in [StateSaveMethod.MEAN, StateSaveMethod.REPLACE]:
                    for mask_sensorial_data in [None, 0.1, UniformScheduler(0, 1, n_epoch), ChoiceScheduler([0,1], n_epoch), 0.5]:
                        for short_time_recall in [{}, {Dimensions.measurement}]:
                        
                            recall_n_past_choices = []
                            recall_stride_choices = []
                            if len(short_time_recall ) > 0:                            
                                recall_n_past_choices = [0, 1, 5]
                                recall_stride_choices = [1,3]
                            
                            for recall_stride in recall_stride_choices:
                                for recall_n_past in recall_n_past_choices:
                                    
                                    recall_n_future_choices = []
                                    if len(short_time_recall) > 0 and recall_n_past > 0:
                                        recall_n_future_choices = [0, 1, 5]
                                    elif len(short_time_recall) > 0:
                                        recall_n_future_choices = [1,5]
                                    
                                    for recall_n_future in recall_n_future_choices:
                                        for positional_encoder_type in ["alibi", "sine"]:
                                            for sensorial_train_losses in [[], [Dimensions.measurement]]:
                                                for block_configuration in [[Dimensions.measurement, Dimensions.measurement],
                                                                            [Dimensions.measurement, Dimensions.STATE_INPUT],
                                                                            [Dimensions.STATE_INPUT, Dimensions.measurement]]:
                                                    for state_activation in ["tanh", "ltanh", None]:
                                                        state_regularizer = None
                                                        if state_activation is None:
                                                            state_regularizer = "mse"

                                                        n_variation += 1

In [9]:
from pydantic import BaseModel, ImportString, create_model
def _make_model(v, name):
    if type(v) is dict:
        return create_model(name, **{k: _make_model(v, k) for k, v in v.items()}), ...
    elif type(v) is type:
        return ImportString, v
    return type(v), v


def make_model(v: dict, name: str):
    return _make_model(v, name)[0]


def save_parameters(n_run: int,
                    base_seed: int,
                    output_dir: str,
                    parameters: dict[str]) -> dict:

    log = {"n_run": n_run,
           "base_seed": base_seed,
           "parameters": parameters}

    model: BaseModel = make_model(log, "ParametersModel").model_validate(log)
    model_json = model.model_dump_json(indent=4)

    file_path = os.path.join(output_dir, "parameters.json")

    with open(file_path, "w") as file:
        file.write(model_json)

    return {"path": file_path}

In [ ]:
n_epoch = 100

n_variation = 0
configurations = {}
for n_segment in [1, 2]:
    fast_forward_choices = [False]
    if n_segment > 1:
        fast_forward_choices = [True, False]

    for fast_forward in fast_forward_choices:
        for stable_state_epochs in [1]:
            for check_input_masks in [True, False]:
                for state_save_method in [StateSaveMethod.MEAN, StateSaveMethod.REPLACE]:
                    for mask_sensorial_data in [UniformScheduler(0, 1, n_epoch)]:
                        for short_time_recall in [set(), {Dimensions.measurement}]:

                            recall_n_past_choices = [0]
                            recall_stride_choices = [0]
                            if len(short_time_recall) > 0:
                                recall_n_past_choices = [0, 1, 5]
                                recall_stride_choices = [1, 3]

                            for recall_stride in recall_stride_choices:
                                for recall_n_past in recall_n_past_choices:

                                    recall_n_future_choices = []
                                    if len(short_time_recall) > 0 and recall_n_past > 0:
                                        recall_n_future_choices = [0, 1, 5]
                                    elif len(short_time_recall) > 0:
                                        recall_n_future_choices = [1, 5]

                                    for recall_n_future in recall_n_future_choices:
                                        for positional_encoder_type in ["alibi"]:
                                            for block_configuration in [[Dimensions.measurement, Dimensions.measurement],
                                                                        [Dimensions.measurement, Dimensions.STATE_INPUT]]:
                                                for state_activation in ["tanh", "ltanh", None]:
                                                    state_regularizer = None
                                                    if state_activation is None:
                                                        state_regularizer = "mse"

                                                    n_variation += 1

                                                    config = {"n_segment": n_segment,
                                                            "fast_forward": fast_forward,
                                                            "stable_state_epochs":stable_state_epochs,
                                                            "check_input_masks": check_input_masks,
                                                            "state_save_method": state_save_method,
                                                            "mask_sensorial_data": mask_sensorial_data,
                                                            "short_time_recall": short_time_recall,
                                                            "recall_stride_past": recall_stride,
                                                            "recall_stride_future": recall_stride,
                                                            "recall_n_past": recall_n_past,
                                                            "recall_n_future": recall_n_future,
                                                            "positional_encoder_type":positional_encoder_type,
                                                            "block_configuration": block_configuration,
                                                            "state_activation": state_activation}

                                                    model: BaseModel = make_model(config, "ParametersModel").model_validate(config)
                                                    model_json = model.model_dump_json(indent=4)
                                                    variation_hash = hash(model_json)

                                                    configurations[variation_hash] = config
                                                    
                                                    

assert len(configurations) == n_variation
n_variation, n_variation/800, (n_variation/800)**0.5

In [23]:
len(configurations)

1152

In [20]:
import numpy as np

_, counts = np.unique(hist, return_counts=True)

counts.max()

1

In [8]:
codes[0]

'1FalseTrueMEAN<world_machine.train.scheduler.UniformScheduler object at 0x00000198292BAC00>{<Dimensions.measurement: 2>}101[<Dimensions.measurement: 2>, <Dimensions.measurement: 2>]tanh'

In [59]:
(30*7*24*60*60)/22757.4

797.2791267895277

In [60]:
n_variation/((30*7*24*60*60)/22757.1333333)

11.559178835961905

In [45]:
with open("test.bin", "wb") as file:
    pickle.dump(configurations, file)

In [46]:
with open("test.bin", "rb") as file:
    configurations2 = pickle.load(file)

In [48]:
configurations2[0]

(1,
 False,
 1,
 True,
 <StateSaveMethod.MEAN: 1>,
 0.5,
 (<Dimensions.measurement: 2>,),
 1,
 0,
 1,
 'alibi',
 (),
 (<Dimensions.measurement: 2>, <Dimensions.measurement: 2>),
 'tanh')